# Load libraries

In [ ]:
import sys
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import scipy.io as io
from scipy.sparse import csr_matrix
import logging
import os
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import scipy.stats as stats
import statsmodels.formula.api as smf
import statsmodels.api as sm
import math
import gc
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from imblearn.over_sampling import SMOTE
import random
from sklearn.utils import shuffle
import combat
import patsy
logging.basicConfig(level=logging.INFO)
savepath = 'Lupus_study_adjusted_V3.h5ad'
nonormpath = 'Lupus_study_nonorm.h5ad'

# Read data

In [ ]:
%matplotlib inline
##################
# Configure file #
##################
sc.settings.verbosity = 2
sc.settings.autoshow = False
sc.settings.set_figure_params(dpi=100, dpi_save=300, format='png', frameon=False, transparent=True, fontsize=16)
plt.rcParams["image.aspect"] = "equal"
adata = sc.read(savepath, cache=True)
print('Processed dataset: {}'.format(adata))
figdir = "./figures."
sc.settings.figdir = "./figures."
MASTERCOLORS = sc.pl.palettes.default_64
MASTERCOLORS.remove("#FEFFE6")
MASTERCOLORS.remove("#FFFF00")


In [ ]:
adata.obs['ind_cov'][adata.obs['ind_cov']=='1771'] = '1771_1771'
adata.obs['ind_cov'][adata.obs['ind_cov']=='1791'] = '1791_1791'
adata.obs['ind_cov'][adata.obs['ind_cov']=='1240'] = '904425200_904425200'
adata.obs['ind_cov'][adata.obs['ind_cov']=='FLARE003'] = '904194200_904194200'
adata.obs['ind_cov'][adata.obs['ind_cov']=='FLARE008'] = '1763_1763'

# Projection

In [ ]:
disease_umap = sc.pl.umap(adata, color=['disease_cov', 'SLE status'], size=1, show=True, edgecolor="none", palette=sc.pl.palettes.vega_20_scanpy, save='.disease.png')

In [ ]:
sc.tl.embedding_density(adata, basis='umap', groupby='disease_cov')
sc.pl.embedding_density(adata, basis='umap', key='umap_density_disease_cov', show=True,  save='umap_density_disease_cov.png')

In [ ]:
sc.tl.embedding_density(adata, basis='umap', groupby='disease_cov')
sc.pl.embedding_density(adata, basis='umap', key='umap_density_disease_cov', show=True,  save='umap_density_disease_cov.png')

In [ ]:
disease_umap = sc.pl.umap(adata, color='batch_cov', size=3,show=True, edgecolor="none", palette=sc.pl.palettes.vega_20_scanpy, save='.batch.png')

## Plot individual variability

In [ ]:
# Plot 16 plots for the individuals.
colors = ["#8ED1C6","#FCF6B5", "#BEBAD9", "#F47F72", "#81B1D3", "#FBB463", "#B4D66C", "#F9CEE1", "#DAD9D9", "#BC80B7", "#CDE6C4", "#FEEC6E", "#E31F26", "#387EB9", "#4EAF49", "#984F9F", "#8ED1C6","#FCF6B5", "#BEBAD9"];
fig,ax = plt.subplots(nrows=4,ncols=4, figsize=(7,7),sharex=True, sharey=True)
fig.tight_layout()
plt.subplots_adjust(wspace=-0.1, hspace=0)

batch_name = adata.obs.batch_cov.unique()[0]
batch = adata.obs.ind_cov_disease_cov[adata.obs.batch_cov==batch_name].unique();
batch = np.sort(batch.categories.values)
for ind_i in list(range(16)):
    ind = batch[ind_i]
    col = colors[ind_i]
    sc.pl.umap(adata[adata.obs.ind_cov_disease_cov==ind,], color = "ind_cov_disease_cov", title=None, palette=[col,col], ax=ax[(ind_i-1)%4,math.floor(ind_i/4)], size=20, edgecolor="none")
    ax[(ind_i-1)%4,math.floor(ind_i/4)].get_xaxis().set_visible(False)
    ax[(ind_i-1)%4,math.floor(ind_i/4)].get_yaxis().set_visible(False)
    ax[(ind_i-1)%4,math.floor(ind_i/4)].get_legend().remove()
    ax[(ind_i-1)%4,math.floor(ind_i/4)].set_title("")
    ax[(ind_i-1)%4,math.floor(ind_i/4)].set_aspect("equal")

fig.savefig(figdir+'/'+batch_name+'.ind.png')

# Rank genes and plot leiden groups

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='t-test', n_genes=25)
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
sc.pl.umap(adata, color='leiden', size=3, show=True, edgecolor="none", save='.leiden.png', palette=MASTERCOLORS)

# Most expressed genes per leiden group

In [ ]:
pd.set_option('display.max_columns', None)
unique_leiden = np.unique(adata.obs['leiden'].values)
# Compile list of top genes
GeneRanks = pd.DataFrame()
for ii in range(len(unique_leiden)):
    GeneRanks[str('leiden_' + str(ii))] = adata.var_names[np.flipud(np.argsort(np.mean(adata.X[adata.obs['leiden'] == str(ii)], axis=0)))]
GeneRanks.to_csv('Flare_study_top_expression.csv')
GeneRanks.head(10)

# Label cell populations

In [ ]:
adata.obs['ct_cov'] = adata.obs['ct_cov'].astype('object')
adata.obs['ct_cov'].loc[adata.obs.leiden == "0"] = "B Cells" ## good                                                                                    
adata.obs['ct_cov'].loc[adata.obs.leiden == "1"] = "Cytotoxic T Cells" ## good                                                                                    
adata.obs['ct_cov'].loc[adata.obs.leiden == "2"] = "Natural Killer Cells" ## good                                                                                     
adata.obs['ct_cov'].loc[adata.obs.leiden == "3"] = "Naive T Cells" ## good                                                                                    
adata.obs['ct_cov'].loc[adata.obs.leiden == "4"] = "Classical Monocytes" ## good    Effector Memory T Cells                                                                                 
adata.obs['ct_cov'].loc[adata.obs.leiden == "5"] = "Classical Monocytes" ## good                                                                                    
adata.obs['ct_cov'].loc[adata.obs.leiden == "6"] = "Classical Monocytes" ## good                                                                                    
adata.obs['ct_cov'].loc[adata.obs.leiden == "7"] = "Cytotoxic T Cells" ## good        Nonclassical Monocytes                                                                            
adata.obs['ct_cov'].loc[adata.obs.leiden == "8"] = "Effector Memory T Cells" ## good     Classical Dendritic Cells                                                                               
adata.obs['ct_cov'].loc[adata.obs.leiden == "9"] = "Classical Monocytes" ## good                                                                                    
adata.obs['ct_cov'].loc[adata.obs.leiden == "10"] = "B Cells" ## good     Megakaryocytes                                                                             
adata.obs['ct_cov'].loc[adata.obs.leiden == "11"] = "Naive T Cells" ## good Plasmacytoid Dendritic Cells                                                                                   
adata.obs['ct_cov'].loc[adata.obs.leiden == "12"] = "Nonclassical Monocytes" ## good                                                                                   
adata.obs['ct_cov'].loc[adata.obs.leiden == "13"] = "Classical Monocytes" ## good  Macro    Proliferating T Cells                                                                             
adata.obs['ct_cov'].loc[adata.obs.leiden == "14"] = "Doublets" ## good                                                                                  
adata.obs['ct_cov'].loc[adata.obs.leiden == "15"] = "Naive T Cells" ## good                                                                                   
adata.obs['ct_cov'].loc[adata.obs.leiden == "16"] = "T-Reg T Cells" ## good                                                                                   
adata.obs['ct_cov'].loc[adata.obs.leiden == "17"] = "Doublets" ## good                                                                                  
adata.obs['ct_cov'].loc[adata.obs.leiden == "18"] = "Cytotoxic T Cells" ## good  
adata.obs['ct_cov'].loc[adata.obs.leiden == "19"] = "Naive T Cells" ## good
adata.obs['ct_cov'].loc[adata.obs.leiden == "20"] = "Naive T Cells" ## good
adata.obs['ct_cov'].loc[adata.obs.leiden == "21"] = "Naive T Cells" ## good
adata.obs['ct_cov'].loc[adata.obs.leiden == "22"] = "Classical Dendritic Cells" ## good
adata.obs['ct_cov'].loc[adata.obs.leiden == "23"] = "Natural Killer Cells" ## good
adata.obs['ct_cov'].loc[adata.obs.leiden == "24"] = "Naive T Cells" ## good
adata.obs['ct_cov'].loc[adata.obs.leiden == "25"] = "Nonclassical Monocytes" ## good
adata.obs['ct_cov'].loc[adata.obs.leiden == "26"] = "Classical Monocytes" ## good
adata.obs['ct_cov'].loc[adata.obs.leiden == "27"] = "Progenitor Cells" ## good
adata.obs['ct_cov'].loc[adata.obs.leiden == "28"] = "Effector Memory T Cells" ## good
adata.obs['ct_cov'].loc[adata.obs.leiden == "29"] = "Proliferating T Cells" ## good
adata.obs['ct_cov'].loc[adata.obs.leiden == "30"] = "RBCs" ## good
adata.obs['ct_cov'].loc[adata.obs.leiden == "31"] = "Classical Monocytes" ## good
adata.obs['ct_cov'].loc[adata.obs.leiden == "32"] = "Plasmacytoid Dendritic Cells" ## good
adata.obs['ct_cov'].loc[adata.obs.leiden == "33"] = "Naive T Cells" ## good
adata.obs['ct_cov'].loc[adata.obs.leiden == "34"] = "B Cells" ## good
adata.obs['ct_cov'].loc[adata.obs.leiden == "35"] = "Naive T Cells" ## good
adata.obs['ct_cov'].loc[adata.obs.leiden == "36"] = "Naive T Cells" ## good
adata.obs['ct_cov'].loc[adata.obs.leiden == "37"] = "RBCs" ## good
adata.obs['ct_cov'].loc[adata.obs.leiden == "38"] = "Naive T Cells" ## good
adata.obs['ct_cov'].loc[adata.obs.leiden == "39"] = "Megakaryocytes" ## good
adata.obs['ct_cov'].loc[adata.obs.leiden == "40"] = "Naive T Cells" ## good
adata.obs['ct_cov'].loc[adata.obs.leiden == "41"] = "B Cells" ## good
adata.obs['ct_cov'] = adata.obs.ct_cov.astype('category')

In [ ]:
sc.pl.umap(adata, color=['CD45RA|PTPRC|j95-28|pAbO', 'CD45RO|PTPRC|j95-19|pAbO', 'CD4|CD4|j95-14|pAbO', 'CD8|CD8A|j95-25|pAbO'], size=10)

In [ ]:
sc.settings.set_figure_params(dpi=100, dpi_save=300, format='png', frameon=False, transparent=True, fontsize=16)
plt.rcParams["image.aspect"] = "equal"
MasterORDER = ['Classical Monocytes','Nonclassical Monocytes', 'Classical Dendritic Cells', 'Plasmacytoid Dendritic Cells', 'Naive T Cells', 'Effector Memory T Cells', 'T-Reg T Cells', 'Cytotoxic T Cells', 'Proliferating T Cells', 'Natural Killer Cells', 'B Cells', 'Progenitor Cells', 'Megakaryocytes', 'RBCs', 'Doublets']
#colorrs = ["#E58606","#5D69B1","#52BCA3","#99C945","#CC61B0","#24796C","#DAA51B","#2F8AC4","#764E9F","#ED645A","#CC3A8E",'#BC23FF', '#D790FF']
colorrs = ["#4E79A7","#A0CBE8","#F28E2B","#FFBE7D","#8CD17D","#B6992D","#499894","#E15759","#FF9D9A","#79706E","#D37295","#FABFD2", '#000000',"#B07AA1","#D4A6C8","#9D7660",
                 "#E58606", "#5D69B1", "#24796C", '#DAA51B', '#000000', '#99C945', '#ED645A']

adata.obs['ct_cov'] = adata.obs['ct_cov'].cat.reorder_categories(MasterORDER)
adata.uns['ct_cov_colors'] = colorrs
celltype_umap = sc.pl.umap(adata, color='ct_cov', show=True, size=3, edgecolor="none")

# Remove RBCs and Doublets from further analysis

In [ ]:
adata = adata[~adata.obs['ct_cov'].isin(['RBCs', 'Doublets'])]

In [ ]:
sc.settings.set_figure_params(dpi=100, dpi_save=300, format='png', frameon=False, transparent=True, fontsize=16)
plt.rcParams["image.aspect"] = "equal"
MasterORDER = ['Classical Monocytes','Nonclassical Monocytes', 'Classical Dendritic Cells', 'Plasmacytoid Dendritic Cells', 'Naive T Cells', 'Effector Memory T Cells', 'T-Reg T Cells','Cytotoxic T Cells', 'Proliferating T Cells', 'Natural Killer Cells', 'B Cells', 'Progenitor Cells', 'Megakaryocytes']
#colorrs = ["#E58606","#5D69B1","#52BCA3","#99C945","#CC61B0","#24796C","#DAA51B","#2F8AC4","#764E9F","#ED645A","#CC3A8E",'#BC23FF', '#D790FF']
colorrs = ["#4E79A7","#A0CBE8","#F28E2B","#FFBE7D","#8CD17D","#B6992D","#499894","#E15759","#FF9D9A","#79706E","#D37295","#FABFD2", '#000000',"#B07AA1","#D4A6C8","#9D7660",
                 "#E58606", "#5D69B1", "#24796C", '#DAA51B', '#000000', '#99C945', '#ED645A']

adata.obs['ct_cov'] = adata.obs['ct_cov'].cat.reorder_categories(MasterORDER)
adata.uns['ct_cov_colors'] = colorrs
celltype_umap = sc.pl.umap(adata, color='ct_cov', show=True, size=3, edgecolor="none")

# Proportions and Statistics

In [ ]:
## Make proportion plots
adata_obs_small = adata.obs
ind_count = adata_obs_small.groupby(['ind_cov_disease_cov','ct_cov','disease_cov','pop_cov', 'ind_cov'])['ct_cov'].count()
ind_count_sums = ind_count.groupby(level=[0]).sum()
ind_count_sums = ind_count_sums.reset_index(name="counts")
ind_perc = ind_count/ind_count.groupby(level=[0]).transform(sum)*100
ind_perc = ind_perc.reset_index(name="ct_perc")
# Add weights to WLS
ind_perc['counts'] = ind_count.values.tolist()

ind_perc['ind_count_sum'] = list(np.zeros(len(ind_count.values.tolist()),dtype=int))
# Add total sums per individual to structure
for ii in range(len(ind_count_sums)):
    ind_perc['ind_count_sum'][ind_perc.ind_cov_disease_cov==ind_count_sums.ind_cov_disease_cov[ii]] = ind_count_sums.counts[ii]


ind_perc.ind_cov_disease_cov = ind_perc.ind_cov_disease_cov.apply(lambda x: x.split('_')[0])
ind_count_sums.ind_cov_disease_cov = ind_count_sums.ind_cov_disease_cov.apply(lambda x: x.split('_')[0])
#ind_perc = ind_perc.set_index('ct_cov').join(cg_cov.set_index('ct_cov'))
ind_perc.ind_cov_disease_cov = ind_perc.ind_cov_disease_cov.astype("str")
ind_perc.reset_index(inplace=True)
ind_perc.ct_cov = ind_perc.ct_cov.astype('category')
ind_perc.ct_cov = ind_perc.ct_cov.cat.reorder_categories(adata.obs.ct_cov.cat.categories.values)

perc_plot = sns.catplot(x='disease_cov', y='ct_perc', order=[ "Healthy", "Managed", "Treated", "Flare"], hue='ct_cov', data=ind_perc, kind='violin', col_order=MasterORDER, col='ct_cov', col_wrap=3, cut=0, dodge=False, aspect=1, sharex=False, sharey=False, palette=colorrs)

for ct_i in list(range(len(MasterORDER))):
    ct = MasterORDER[ct_i]
    sns.swarmplot(x="disease_cov", y="ct_perc", data=ind_perc[ind_perc.ct_cov == ct], order=[ "Healthy", "Managed", "Treated", "Flare"], color="0", size=4, ax=perc_plot.axes[ct_i])
    try:
        sns.pointplot(x="disease_cov", y="ct_perc", hue="ind_cov", data=ind_perc[ind_perc.ct_cov == ct], order=[ "Healthy", "Managed", "Treated", "Flare"], color="0", scale=0.3, ax=perc_plot.axes[ct_i])
    except:
        continue
    perc_plot.axes[ct_i].get_yaxis().label.set_visible(False)
    perc_plot.axes[ct_i].get_xaxis().label.set_visible(False)
    perc_plot.axes[ct_i].get_legend().remove()
    perc_plot.set_xticklabels(rotation=90)
    perc_plot.fig.subplots_adjust(wspace=2, hspace = 1)
    
perc_plot.savefig(figdir+"/violin.ct_cov_figure6b.png")
perc_plot.savefig(figdir+"/violin.ct_cov_figure6b.pdf")

# Statistical tests...
print('as a proportion of total PBMC: Healthy vs. Managed WEIGHTED BY TOTAL PBMC COUNTS')
all_out = pd.DataFrame();
ind_perc0 = ind_perc[ind_perc.disease_cov.isin(['Healthy', 'Managed'])]
HEALTHYMANAGED_effect = {}
for ct_i in list(range(len(ind_perc0.ct_cov.cat.categories))):
    ct = ind_perc0.ct_cov.cat.categories[ct_i]
    ct_perc = ind_perc0.ct_perc[ind_perc0.ct_cov==ct]
    weights = ind_perc0.ind_count_sum[ind_perc0.ct_cov==ct]
    disease = ind_perc0.disease_cov[ind_perc0.ct_cov==ct]
    disease = disease.astype("str")
    disease.values[disease=="Managed"] = 0
    disease.values[disease=="Healthy"] = 1
    disease = sm.add_constant(disease)
    est=sm.WLS(ct_perc.astype(float), disease.astype(float), weights=weights)
    #est=sm.OLS(ct_perc.astype(float), disease.astype(float))
    est=est.fit()
    effect=est.params[1]
    pval=est.pvalues[1]
    HEALTHYMANAGED_effect[ct] = effect
    all_out = all_out.append(({"Cell":str(ct),"Beta":str(effect),"Pval":str(pval)}), ignore_index=True)
    all_out['Pval'] = all_out['Pval'].astype(float)
display(all_out)

# Statistical tests...
print('as a proportion of total PBMC: Healthy vs. Treated WEIGHTED BY TOTAL PBMC COUNTS')
all_out = pd.DataFrame();
ind_perc0 = ind_perc[ind_perc.disease_cov.isin(['Healthy', 'Treated'])]
HEALTHYTREATED_effect = {}
for ct_i in list(range(len(ind_perc0.ct_cov.cat.categories))):
    ct = ind_perc0.ct_cov.cat.categories[ct_i]
    ct_perc = ind_perc0.ct_perc[ind_perc0.ct_cov==ct]
    weights = ind_perc0.ind_count_sum[ind_perc0.ct_cov==ct]
    disease = ind_perc0.disease_cov[ind_perc0.ct_cov==ct]
    disease = disease.astype("str")
    disease.values[disease=="Treated"] = 0
    disease.values[disease=="Healthy"] = 1
    disease = sm.add_constant(disease)
    est=sm.WLS(ct_perc.astype(float), disease.astype(float), weights=weights)
    #est=sm.OLS(ct_perc.astype(float), disease.astype(float))
    est=est.fit()
    effect=est.params[1]
    pval=est.pvalues[1]
    HEALTHYTREATED_effect[ct] = effect
    all_out = all_out.append(({"Cell":str(ct),"Beta":str(effect),"Pval":str(pval)}), ignore_index=True)
    all_out['Pval'] = all_out['Pval'].astype(float)
display(all_out)

# Statistical tests...
print('as a proportion of total PBMC: Healthy vs. Flare WEIGHTED BY TOTAL PBMC COUNTS')
all_out = pd.DataFrame();
ind_perc0 = ind_perc[ind_perc.disease_cov.isin(['Healthy', 'Flare'])]
HEALTHYFLARE_effect = {}
HEALTHYFLARE_FC = {}
for ct_i in list(range(len(ind_perc0.ct_cov.cat.categories))):
    ct = ind_perc0.ct_cov.cat.categories[ct_i]
    ct_perc = ind_perc0.ct_perc[ind_perc0.ct_cov==ct]
    weights = ind_perc0.ind_count_sum[ind_perc0.ct_cov==ct]
    disease = ind_perc0.disease_cov[ind_perc0.ct_cov==ct]
    disease = disease.astype("str")
    disease.values[disease=="Flare"] = 0
    disease.values[disease=="Healthy"] = 1
    disease = sm.add_constant(disease)
    est=sm.WLS(ct_perc.astype(float), disease.astype(float), weights=weights)
    #est=sm.OLS(ct_perc.astype(float), disease.astype(float))
    est=est.fit()
    effect=est.params[1]
    pval=est.pvalues[1]
    HEALTHYFLARE_effect[ct] = effect
    HEALTHYFLARE_FC[ct] = math.log2(sum(est.params)/est.params[0])
    all_out = all_out.append(({"Cell":str(ct),"Beta":str(effect),"Pval":str(pval)}), ignore_index=True)
    all_out['Pval'] = all_out['Pval'].astype(float)
display(all_out)

# Statistical tests...
print('as a proportion of total PBMC: Treated vs. Flare WEIGHTED BY TOTAL PBMC COUNTS')
all_out = pd.DataFrame();
ind_perc0 = ind_perc[ind_perc.disease_cov.isin(['Treated', 'Flare'])]
TREATEDFLARE_effect = {}
for ct_i in list(range(len(ind_perc0.ct_cov.cat.categories))):
    ct = ind_perc0.ct_cov.cat.categories[ct_i]
    ct_perc = ind_perc0.ct_perc[ind_perc0.ct_cov==ct]
    weights = ind_perc0.ind_count_sum[ind_perc0.ct_cov==ct]
    disease = ind_perc0.disease_cov[ind_perc0.ct_cov==ct]
    disease = disease.astype("str")
    disease.values[disease=="Flare"] = 0
    disease.values[disease=="Treated"] = 1
    disease = sm.add_constant(disease)
    est=sm.WLS(ct_perc.astype(float), disease.astype(float), weights=weights)
    #est=sm.OLS(ct_perc.astype(float), disease.astype(float))
    est=est.fit()
    effect=est.params[1]
    pval=est.pvalues[1]
    TREATEDFLARE_effect[ct] = effect
    all_out = all_out.append(({"Cell":str(ct),"Beta":str(effect),"Pval":str(pval)}), ignore_index=True)
    all_out['Pval'] = all_out['Pval'].astype(float)
display(all_out)

# Statistical tests...
print('as a proportion of total PBMC: Treated vs. Managed WEIGHTED BY TOTAL PBMC COUNTS')
all_out = pd.DataFrame();
ind_perc0 = ind_perc[ind_perc.disease_cov.isin(['Treated', 'Managed'])]
TREATEDMANAGED_effect = {}
for ct_i in list(range(len(ind_perc0.ct_cov.cat.categories))):
    ct = ind_perc0.ct_cov.cat.categories[ct_i]
    ct_perc = ind_perc0.ct_perc[ind_perc0.ct_cov==ct]
    weights = ind_perc0.ind_count_sum[ind_perc0.ct_cov==ct]
    disease = ind_perc0.disease_cov[ind_perc0.ct_cov==ct]
    disease = disease.astype("str")
    disease.values[disease=="Managed"] = 0
    disease.values[disease=="Treated"] = 1
    disease = sm.add_constant(disease)
    est=sm.WLS(ct_perc.astype(float), disease.astype(float), weights=weights)
    #est=sm.OLS(ct_perc.astype(float), disease.astype(float))
    est=est.fit()
    effect=est.params[1]
    pval=est.pvalues[1]
    TREATEDMANAGED_effect[ct] = effect
    all_out = all_out.append(({"Cell":str(ct),"Beta":str(effect),"Pval":str(pval)}), ignore_index=True)
    all_out['Pval'] = all_out['Pval'].astype(float)
display(all_out)

# Statistical tests...
print('as a proportion of total PBMC: Healthy vs. SLE WEIGHTED BY TOTAL PBMC COUNTS')
all_out = pd.DataFrame();
ind_perc0 = ind_perc[ind_perc.disease_cov.isin(['Healthy', 'Treated', 'Untreated', 'Managed'])]
for ct_i in list(range(len(ind_perc0.ct_cov.cat.categories))):
    ct = ind_perc0.ct_cov.cat.categories[ct_i]
    ct_perc = ind_perc0.ct_perc[ind_perc0.ct_cov==ct]
    weights = ind_perc0.ind_count_sum[ind_perc0.ct_cov==ct]
    disease = ind_perc0.disease_cov[ind_perc0.ct_cov==ct]
    disease = disease.astype("str")
    disease.values[disease!="Healthy"] = 0
    disease.values[disease=="Healthy"] = 1
    disease = sm.add_constant(disease)
    est=sm.WLS(ct_perc.astype(float), disease.astype(float), weights=weights)
    #est=sm.OLS(ct_perc.astype(float), disease.astype(float))
    est=est.fit()
    effect=est.params[1]
    pval=est.pvalues[1]
    all_out = all_out.append(({"Cell":str(ct),"Beta":str(effect),"Pval":str(pval)}), ignore_index=True)
    all_out['Pval'] = all_out['Pval'].astype(float)
display(all_out)

# Statistical tests...
print('as a proportion of total PBMC: Healthy vs. Treated&Flare WEIGHTED BY TOTAL PBMC COUNTS')
all_out = pd.DataFrame();
ind_perc0 = ind_perc[ind_perc.disease_cov.isin(['Healthy', 'Treated', 'Untreated'])]
for ct_i in list(range(len(ind_perc0.ct_cov.cat.categories))):
    ct = ind_perc0.ct_cov.cat.categories[ct_i]
    ct_perc = ind_perc0.ct_perc[ind_perc0.ct_cov==ct]
    weights = ind_perc0.ind_count_sum[ind_perc0.ct_cov==ct]
    disease = ind_perc0.disease_cov[ind_perc0.ct_cov==ct]
    disease = disease.astype("str")
    disease.values[disease!="Healthy"] = 0
    disease.values[disease=="Healthy"] = 1
    disease = sm.add_constant(disease)
    est=sm.WLS(ct_perc.astype(float), disease.astype(float), weights=weights)
    #est=sm.OLS(ct_perc.astype(float), disease.astype(float))
    est=est.fit()
    effect=est.params[1]
    pval=est.pvalues[1]
    all_out = all_out.append(({"Cell":str(ct),"Beta":str(effect),"Pval":str(pval)}), ignore_index=True)
    all_out['Pval'] = all_out['Pval'].astype(float)
display(all_out)


In [ ]:
def getproportions(celltype, group1, group2):
    print('{} proportion for celltype: {}: {}' .format(group1, celltype, ind_perc[ind_perc.disease_cov==group1][ind_perc.ct_cov==celltype].ct_perc.mean()))
    print('{} proportion for celltype: {}: {}' .format(group2, celltype, ind_perc[ind_perc.disease_cov==group2][ind_perc.ct_cov==celltype].ct_perc.mean()))

getproportions(celltype='Classical Monocytes',group1='Healthy', group2='Flare')

# Get differential gene expression for each cell population in psuedobulk

In [ ]:
def get_pseudobulk(adata):
    genes = adata.raw.var_names.tolist()
    people = np.unique(adata.obs['ind_cov'].tolist())
    pseudobulk = pd.DataFrame(index=genes, columns=people)
    for ii in range(len(people)):
        fdata = adata[adata.obs['ind_cov']==people[ii]]
        pseudobulk[people[ii]] = np.ravel(np.sum(fdata.raw.X, axis=0)/len(fdata))
    return pseudobulk

def compute_LogFC(pseudobulk, SLE_people, healthy_people):
    logFC = {}; pval  = {}
    genes = pseudobulk.transpose().keys().tolist()
    for ii in range(len(genes)):
        h = pseudobulk[healthy_people].transpose()[genes[ii]].values
        s = pseudobulk[SLE_people].transpose()[genes[ii]].values
        _, p = stats.ranksums(s,h)
        pval[genes[ii]] = p
        s = np.nanmean(s)
        h = np.nanmean(h)
        logFC[genes[ii]] = np.log2(np.divide(s+10**-3,h+10**-3))
    return logFC, pval

def make_volcanoplot(logFC, pval, title):
    num_std = 6 # Number of standard deviations above/below mean for gene log2 FC cutoff.
    pval_cutoff = 9 # 10^-n (0.001/~39000) allows for 1/1000 false alarm rate corrected for # genes
    s = np.std(np.asarray(list(logFC.values())))
    upper = np.mean(np.asarray(list(logFC.values()))) + (np.std(np.asarray(list(logFC.values())))*num_std)
    lower = np.mean(np.asarray(list(logFC.values()))) - (np.std(np.asarray(list(logFC.values())))*num_std)
    genes = np.asarray(list(logFC.keys()))
    x = np.asarray(list(logFC.values()))
    y = np.log10(list(pval.values()))*-1
    sc.settings.set_figure_params(dpi=150, dpi_save=300, format='png', frameon=False, transparent=True, fontsize=12)
    # Plot non-significant values
    sns.scatterplot(x[(x<=upper) & (x>=lower) & (y<=pval_cutoff)], y[(x<=upper) & (x>=lower) & (y<=pval_cutoff)], color=[0.5, 0.5, 0.5], alpha=0.2)
    sns.scatterplot(x[(x<upper) & (y>pval_cutoff)], y[(x<upper) & (y>pval_cutoff)], color=[0.5, 0.5, 0.5], alpha=0.2)
    sns.scatterplot(x[(x>lower) & (y>pval_cutoff)], y[(x>lower) & (y>pval_cutoff)], color=[0.5, 0.5, 0.5], alpha=0.2)
    # Plot significant values
    sns.scatterplot(x[(x>upper) & (y>pval_cutoff)], y[(x>upper) & (y>pval_cutoff)], color='#a1caf1')
    sns.scatterplot(x[(x<lower) & (y>pval_cutoff)], y[(x<lower) & (y>pval_cutoff)], color='#a1caf1')
    plt.xlabel('Log2 Fold Change')
    plt.ylabel('-log10 p-value')
    plt.title('Volcano plot: {}'.format(title))
    for ii in range(len(genes[(x>upper) & (y>pval_cutoff)])):
        plt.text(x[(x>upper) & (y>pval_cutoff)][ii], y[(x>upper) & (y>pval_cutoff)][ii], s=genes[(x>upper) & (y>pval_cutoff)][ii], fontsize=5)

    for ii in range(len(genes[(x<lower) & (y>pval_cutoff)])):
        plt.text(x[(x<lower) & (y>pval_cutoff)][ii], y[(x<lower) & (y>pval_cutoff)][ii], s=genes[(x<lower) & (y>pval_cutoff)][ii], fontsize=5)
    
    URG = genes[(x>upper) & (y>pval_cutoff)]
    DRG = genes[(x<lower) & (y>pval_cutoff)]
    return URG, DRG
    

## Log2 FC for Total PBMC psuedobulk

In [ ]:
def get_totalPBMC_log2FC(adata):
    healthy_people = np.unique(adata.obs['ind_cov'][adata.obs['SLE status']=='Healthy'].tolist())
    SLE_people = np.unique(adata.obs['ind_cov'][adata.obs['SLE status']=='SLE'].tolist())
    pseudobulk_totalPBMC = get_pseudobulk(adata)
    logFC_totalPBMC, pval_totalPBMC = compute_LogFC(pseudobulk_totalPBMC, SLE_people, healthy_people)
    data = pd.DataFrame(list(logFC_totalPBMC.values()), index=list(logFC_totalPBMC.keys()), columns=['Log2FC'])
    data2 = pd.DataFrame(list(pval_totalPBMC.values()), index=list(pval_totalPBMC.keys()), columns=['p-value'])
    data = pd.concat([data, data2], axis=1)
    data = data.transpose()
    data.to_csv('Log2FC-SLEvsHealthy-totalPBMC.csv')
    return logFC_totalPBMC, pval_totalPBMC

#logFC_totalPBMC, pval_totalPBMC = get_totalPBMC_log2FC(adata)

tmp = pd.read_csv('Log2FC-SLEvsHealthy-totalPBMC.csv', index_col=0)
logFC_totalPBMC = tmp.transpose()['Log2FC'].to_dict()
pval_totalPBMC = tmp.transpose()['p-value'].to_dict()

In [ ]:
URG_total, DRG_total = make_volcanoplot(logFC_totalPBMC, pval_totalPBMC, title='Total PBMCs')

# Log2 FC for each subpopulation

In [ ]:
def run_subpop_log2FC(adata):
    ct_groups = np.unique(adata.obs['ct_cov'].tolist())
    logFC_groups = {}; pval_groups = {};
    for ii in range(len(ct_groups)):
        print('Cell type: {}'.format(ct_groups[ii]))
        fdata = adata[adata.obs['ct_cov']==ct_groups[ii]]
        healthy_people = np.unique(fdata.obs['ind_cov'][fdata.obs['SLE status']=='Healthy'].tolist())
        SLE_people = np.unique(fdata.obs['ind_cov'][fdata.obs['SLE status']=='SLE'].tolist())
        pseudobulk = get_pseudobulk(fdata)
        logFC, pval = compute_LogFC(pseudobulk, SLE_people, healthy_people)
        logFC_groups[ct_groups[ii]] = logFC
        pval_groups[ct_groups[ii]] = pval
        data = pd.DataFrame(list(logFC_groups[ct_groups[ii]].values()), index=list(logFC_groups[ct_groups[ii]].keys()), columns=['Log2FC'])
        data2 = pd.DataFrame(list(pval_groups[ct_groups[ii]].values()), index=list(pval_groups[ct_groups[ii]].keys()), columns=['p-value'])
        data = pd.concat([data, data2], axis=1)
        data = data.transpose()
        data.to_csv('Log2FC-SLEvsHealthy-{}.csv'.format(ct_groups[ii]))
        
def load_subpop_log2FC(adata):
    ct_groups = np.unique(adata.obs['ct_cov'].tolist())
    logFC_groups = {}; pval_groups = {};
    for ii in range(len(ct_groups)):
        tmp = pd.read_csv('Log2FC-SLEvsHealthy-{}.csv'.format(ct_groups[ii]), index_col=0)
        logFC_groups[ct_groups[ii]] = tmp.transpose()['Log2FC'].to_dict()
        pval_groups[ct_groups[ii]] = tmp.transpose()['p-value'].to_dict()
    return logFC_groups, pval_groups
    
#run_subpop_log2FC(adata)

logFC_groups, pval_groups = load_subpop_log2FC(adata)


In [ ]:
URG = {}; DRG = {}; # Up and Down Regulated Genes for each population
ct_groups = np.unique(adata.obs['ct_cov'].tolist())
for ii in range(len(ct_groups)):
    plt.figure()
    URG[ct_groups[ii]], DRG[ct_groups[ii]] = make_volcanoplot(logFC_groups[ct_groups[ii]], pval_groups[ct_groups[ii]], ct_groups[ii])
    

# Compute Gene-Cell-Type specific expression per individual

In [ ]:
pseudobulk = get_pseudobulk(adata)
corr_matrix = pd.DataFrame(index=list(pseudobulk.columns))
for ii in range(len(URG_total)):
    corr_matrix['{}-URG-totalPBMC'.format(URG_total[ii])] = pseudobulk.transpose()[URG_total[ii]].values
for ii in range(len(DRG_total)):
    corr_matrix['{}-DRG-totalPBMC'.format(DRG_total[ii])] = pseudobulk.transpose()[DRG_total[ii]].values
    
ct_groups = np.unique(adata.obs['ct_cov'].tolist())
for ii in range(len(ct_groups)):
    try:
        print('Cell type: {}'.format(ct_groups[ii]))
        fdata = adata[adata.obs['ct_cov']==ct_groups[ii]]
        pseudobulk = get_pseudobulk(fdata)
        for gg in range(len(URG[ct_groups[ii]])):   
            corr_matrix['{0}-URG-{1}'.format(URG[ct_groups[ii]][gg], ct_groups[ii])] = pseudobulk.transpose()[URG[ct_groups[ii]][gg]].values
        for gg in range(len(DRG[ct_groups[ii]])):   
            corr_matrix['{0}-DRG-{1}'.format(DRG[ct_groups[ii]][gg], ct_groups[ii])] = pseudobulk.transpose()[DRG[ct_groups[ii]][gg]].values
    except:
        continue
corr_matrix.to_csv('Expression-Composition-corrmatrix.csv')

# Make signatures for cell type specific expression without pan signature

In [ ]:
'''
ct_groups = np.unique(adata.obs['ct_cov'].tolist())
URG_exclude = {}; DRG_exclude = {}
for ii in range(len(ct_groups)):
    listURG = list(URG[ct_groups[ii]])
    listDRG = list(DRG[ct_groups[ii]])
    removallist = list(set(URG[ct_groups[ii]]).intersection(URG_total))
    removallistD = list(set(DRG[ct_groups[ii]]).intersection(DRG_total))
    for item in removallist:
        listURG.remove(item)
    for item in removallistD:
        listDRG.remove(item)
    URG_exclude[ct_groups[ii]] = listURG
    DRG_exclude[ct_groups[ii]] = listDRG
'''

# Compute averaged gene expression of significant genes per person 

In [ ]:
'''
pseudobulk = get_pseudobulk(adata)
corr_matrix = pd.DataFrame(index=list(pseudobulk.columns))
corr_matrix['URG-totalPBMC'] = pseudobulk.transpose()[URG_total].mean(axis=1).values
corr_matrix['DRG-totalPBMC'] = pseudobulk.transpose()[DRG_total].mean(axis=1).values


ct_groups = np.unique(adata.obs['ct_cov'].tolist())
for ii in range(len(ct_groups)):
    try:
        print('Cell type: {}'.format(ct_groups[ii]))
        fdata = adata[adata.obs['ct_cov']==ct_groups[ii]]
        pseudobulk = get_pseudobulk(fdata)
        #corr_matrix['URG-{}'.format(ct_groups[ii])] = pseudobulk.transpose()[URG[ct_groups[ii]]].mean(axis=1).values
        #corr_matrix['DRG-{}'.format(ct_groups[ii])] = pseudobulk.transpose()[DRG[ct_groups[ii]]].mean(axis=1).values
        corr_matrix['URG-{}'.format(ct_groups[ii])] = pseudobulk.transpose()[URG_exclude[ct_groups[ii]]].mean(axis=1).values
        corr_matrix['DRG-{}'.format(ct_groups[ii])] = pseudobulk.transpose()[DRG_exclude[ct_groups[ii]]].mean(axis=1).values
    except:
        continue
corr_matrix.to_csv('Expression-Composition-corrmatrix.csv')
'''

In [ ]:
corr_matrix = pd.read_csv('Expression-Composition-corrmatrix.csv', index_col=0)

In [ ]:
pd.set_option('display.max_columns', None)
display(corr_matrix)
# Clustermap this gene expression
# Average pan expression genes across cell types and within cell types

In [ ]:
# genes by cell type + total PBMC corr matrix. Be sure to average the people after pseudobulk. Split by healthy and SLE

In [ ]:
# Corrlation plot of corrmatrix expression values. Combine over pan gene PBMC and per cell type
sc.settings.set_figure_params(dpi=250, dpi_save=300, format='png', frameon=False, transparent=True, fontsize=12)
sns.set(font_scale=0.2)
sns.clustermap(corr_matrix.dropna().transpose(), standard_scale=0)

# Compute composition of each cell type per person

In [ ]:
## Make proportion plots
adata_obs_small = adata.obs
ind_count = adata_obs_small.groupby(['ind_cov', 'ct_cov','SLE status'])['ct_cov'].count()
ind_count_sums = ind_count.groupby(level=[0]).sum()
ind_count_sums = ind_count_sums.reset_index(name="counts")
ind_perc = ind_count/ind_count.groupby(level=[0]).transform(sum)*100
ind_perc = ind_perc.reset_index(name="ct_perc")


In [ ]:
# Add composition changes
people = corr_matrix.index.tolist()
ct_groups = np.unique(adata.obs['ct_cov'].tolist())
for jj in range(len(ct_groups)):
    tmp = ind_perc[ind_perc['ct_cov']==ct_groups[jj]]
    tmplist = np.empty((len(people),))
    tmplist[:] = np.nan
    for ii in range(len(people)):
        try:
            tmplist[ii] = tmp[tmp['ind_cov']==people[ii]]['ct_perc'].values[0]
        except:
            continue
    corr_matrix['Proportion-{}'.format(ct_groups[jj])] = tmplist
    
# Remove empty categories    
labels = list(corr_matrix.columns)
for lbl in labels:
    if np.sum(np.isnan(corr_matrix[lbl]))==len(corr_matrix):
        print('Removing category: {} due to lack of data.'.format(lbl))
        corr_matrix = corr_matrix.drop(lbl, axis='columns')

# Compute correlations between URG, DRGs and cell type composition

In [ ]:
correlations = corr_matrix.corr(method='spearman')
correlations[np.isnan(correlations)] = 0
sns.set(font_scale=0.2)
sns.clustermap(np.abs(correlations))
plt.savefig("Correlation_plot.png", bbox_inches='tight')

In [ ]:
def plot_barplot(correlations, label):
    sc.settings.set_figure_params(dpi=150, dpi_save=200, format='png', frameon=False, transparent=True, fontsize=5)
    x = np.asarray(list((correlations[label].values)))
    indices = np.flipud(np.argsort(x)); x = x[indices];
    y = np.asarray(list(correlations[label].index)); y = y[indices]
    x = np.concatenate((x[:20], x[-20:]))
    y = np.concatenate((y[:20], y[-20:]))
    g = sns.barplot(x, y)
    plt.title('Correlations to {}'.format(label))

def plot_gene_expression(cdata, celltypes, gene, normalize):
    sns.set(font_scale=1)
    indlist  = cdata.obs['ind_cov_disease_cov'].unique().tolist()
    ind_id = []; dz_id = []; ct_id = []; values = []; count = []
    for celltype in celltypes:
        for ii in range(len(indlist)):
            values.append(cdata.raw.X[(cdata.obs['ct_cov'].values==celltype) & (cdata.obs['ind_cov_disease_cov'].values==indlist[ii]), cdata.raw.var_names==gene].mean())
            count.append(cdata.raw.X[cdata.obs['ind_cov_disease_cov'].values==indlist[ii], cdata.raw.var_names==gene].shape[1])
            normvalues = np.divide(np.asarray(values), np.asarray(count)) # Normalize by cell number
            ind_id.append(indlist[ii])
            dz_id.append(cdata.obs['disease_cov'][cdata.obs['ind_cov_disease_cov']==indlist[ii]].values.unique()[0])
            ct_id.append(celltype)
    if normalize == True:
        genexpression = pd.DataFrame(data={'Unique_ID': ind_id, 'disease_cov': dz_id, gene: normvalues, 'Cell_Count':count, 'ct_cov':ct_id})    
    else:
        genexpression = pd.DataFrame(data={'Unique_ID': ind_id, 'disease_cov': dz_id, gene: values, 'Cell_Count':count, 'ct_cov':ct_id})    
    genexpression.ct_cov = genexpression.ct_cov.astype('category')
    perc_plot = sns.catplot(x='disease_cov', y=gene, order=[ "Healthy", "Managed", "Treated", "Flare"], hue='ct_cov', data=genexpression, kind='violin', col='ct_cov', col_wrap=2, cut=0, dodge=False, aspect=1, sharex=False, sharey=True)
    for ct_i in range(len(celltypes)):
        sns.swarmplot(x="disease_cov", y=gene, data=genexpression[genexpression.ct_cov==celltypes[ct_i]], order=[ "Healthy", "Managed", "Treated", "Flare"], color="0", size=6, ax=perc_plot.axes[ct_i])
        try:
            sns.pointplot(x="disease_cov", y=gene, hue="Unique_ID", data=genexpression[genexpression.Cell_Type==celltypes[ct_i]], order=[ "Healthy", "Managed", "Treated", "Flare"], color="0", scale=0.3, ax=perc_plot.axes[ct_i])
        except:
            continue
        perc_plot.axes[ct_i].get_xaxis().label.set_visible(False)
        perc_plot.axes[ct_i].get_legend().remove()
        perc_plot.set_xticklabels(rotation=90)
        perc_plot.fig.subplots_adjust(wspace=0.5, hspace = 1)

In [ ]:
plot_barplot(correlations, label='ISG15-URG-totalPBMC')

In [ ]:
plot_barplot(correlations, label='Proportion-Naive T Cells')

In [ ]:
plot_barplot(correlations, label='NOG-DRG-totalPBMC')

In [ ]:
plot_barplot(correlations, label='Proportion-Plasmacytoid Dendritic Cells')

In [ ]:
plot_barplot(correlations, label='Proportion-Classical Dendritic Cells')

In [ ]:
plot_barplot(correlations, label='Proportion-Classical Monocytes')

In [ ]:
plot_barplot(correlations, label='Proportion-Cytotoxic T Cells')

In [ ]:
plot_barplot(correlations, label='Proportion-Proliferating T Cells')

In [ ]:
ct_groups = np.unique(adata.obs['ct_cov'].tolist())
genes = ['ISG15', 'IFI6', 'NOG']
for gene in genes:
    plot_gene_expression(adata, celltypes=['Classical Monocytes', 'Naive T Cells'], gene=gene, normalize=True)

In [ ]:
ct_groups = np.unique(adata.obs['ct_cov'].tolist())
genes = ['ISG15', 'IFI6', 'NOG']
for gene in genes:
    plot_gene_expression(adata, celltypes=['Classical Monocytes', 'Naive T Cells'], gene=gene, normalize=False)

# Clinical covariate analysis

In [ ]:
clincal = pd.read_csv('lupus.sledai.txt', sep='\t')
# Add study disease status
disease_status = {}
for ii in range(len(clincal['genotypeid'])):
    try:
        disease_status[clincal['genotypeid'][ii]] = 1 #np.unique(adata.obs['SLE status'][adata.obs['ind_cov']==clincal['genotypeid'][ii]])[0]
    except:
        disease_status[clincal['genotypeid'][ii]] = 1 #'SLE' # This person by definition of being in sheet has SLE
        #print('Individual: {} not in study data'.format(clincal['genotypeid'][ii]))

clincal['Disease_status'] = disease_status.values()
display(clincal)

### Remove SLE patients that have no clinical data

In [ ]:
corr_matrix_mod = corr_matrix[~corr_matrix.transpose().columns.isin(['FLARE002', 'FLARE004', 'FLARE005', 'FLARE006', 
                                                                     'FLARE007', 'FLARE009', 'FLARE011', 'FLARE013', 
                                                                     'FLARE014', 'FLARE015', 'FLARE016', 'FLARE017', 
                                                                     'FLARE018', 'FLARE019', 'FLARE020'])]

In [ ]:
def build_covar(corr_matrix_mod, keys):
    clincal = pd.read_csv('lupus.sledai.txt', sep='\t')
    inds = list(corr_matrix_mod.transpose().columns)
    # Add study disease status
    disease_status = {}
    for ii in range(len(clincal['genotypeid'])):
        try:
            disease_status[clincal['genotypeid'][ii]] = 1 #np.unique(adata.obs['SLE status'][adata.obs['ind_cov']==clincal['genotypeid'][ii]])[0]
        except:
            disease_status[clincal['genotypeid'][ii]] = 1 #'SLE' # This person by definition of being in sheet has SLE
            #print('Individual: {} not in study data'.format(clincal['genotypeid'][ii]))

    clincal['Disease_status'] = disease_status.values()
    for key in keys:
        covar = {}
        for ii in range(len(inds)):
            if np.sum(clincal['genotypeid'].isin([inds[ii]]).values):
                covar[inds[ii]] = clincal[clincal['genotypeid'].isin([inds[ii]]).values][key].values[0]
            else:
                covar[inds[ii]] = 0
        corr_matrix_mod[key] = list(covar.values())
    return corr_matrix_mod

In [ ]:
corr_matrix_mod = build_covar(corr_matrix_mod, keys = ['SLAQ score', 'sleactivity', 'SLIC score', 'acrcsum', 'sledaiscore'])
display(corr_matrix_mod)

In [ ]:
correlations = corr_matrix_mod.corr(method='spearman')
correlations[np.isnan(correlations)] = 0
sns.set(font_scale=0.2)
sns.clustermap(np.abs(correlations))
plt.savefig("Correlation+clinical_plot.png", bbox_inches='tight')

In [ ]:
plot_barplot(correlations, label='SLAQ score')

In [ ]:
plot_barplot(correlations, label='sleactivity')

In [ ]:
plot_barplot(correlations, label='SLIC score')

In [ ]:
plot_barplot(correlations, label='sledaiscore')

In [ ]:
plot_barplot(correlations, label='acrcsum')

In [ ]:
sns.set(font_scale=1)
sns.regplot(corr_matrix_mod['NOG-DRG-totalPBMC'].values, corr_matrix_mod['SLAQ score'].values)
#sns.regplot(corr_matrix_mod['NOG-DRG-Naive T Cells'].values, corr_matrix_mod['SLAQ score'].values)
plt.xlabel('NOG expression')
plt.ylabel('SLAQ score')

In [ ]:
sns.set(font_scale=1)
sns.regplot(corr_matrix_mod['NOG-DRG-totalPBMC'].values, corr_matrix_mod['acrcsum'].values)
#sns.regplot(corr_matrix_mod['NOG-DRG-Naive T Cells'].values, corr_matrix_mod['SLAQ score'].values)
plt.xlabel('NOG expression')
plt.ylabel('SLAQ score')

# Logistic regression on clinical variables

In [ ]:
def logistic_regression(corr_matrix, clincal_var, n_boot):
    # Remove people with no clinical variables
    corr_matrix_logreg = corr_matrix[~corr_matrix.transpose().columns.isin(['FLARE002', 'FLARE004', 'FLARE005', 'FLARE006', 
                                                                         'FLARE007', 'FLARE009', 'FLARE011', 'FLARE013', 
                                                                         'FLARE014', 'FLARE015', 'FLARE016', 'FLARE017', 
                                                                         'FLARE018', 'FLARE019', 'FLARE020'])]
    # Add clinical outcome
    corr_matrix_logreg = build_covar(corr_matrix_logreg, keys = [clincal_var])
    # Drop NaN from dataframe
    corr_matrix_logreg = corr_matrix_logreg.dropna()
    # Dependent variable
    Y = corr_matrix_logreg[clincal_var].values.ravel()
    # Remove dependent variable from variables
    corr_matrix_logreg = corr_matrix_logreg.drop(clincal_var, axis=1)
    # observations x features
    X = corr_matrix_logreg.as_matrix()
    # Variables of interest
    VOIs = {}; ROC_AUC = {}; FPR = {}; TPR = {}
    for ii in range(n_boot):
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

        # Over sample cases and undersample controls to improve classification
        os = SMOTE(random_state=0)
        os_data_X, os_data_y = os.fit_sample(X_train, y_train)

        # Setup logistic regression
        logreg = LogisticRegression(max_iter=5000, penalty='elasticnet', solver='saga', l1_ratio=0.5)
        # Find optimal parameters
        #rfe = RFE(logreg, 3)
        rfe = SelectFromModel(logreg, threshold='10*mean')
        rfe = rfe.fit(os_data_X, os_data_y)
        ind_var = np.asarray(list(corr_matrix_logreg.columns))

        # Preserve optimate features only from train and test set
        #os_data_X = os_data_X[:, corr_matrix_logreg.columns.isin(ind_var[rfe.ranking_==1])]
        #X_test = X_test[:, corr_matrix_logreg.columns.isin(ind_var[rfe.ranking_==1])]
        os_data_X = os_data_X[:, rfe.get_support()]
        X_test = X_test[:, rfe.get_support()]
        # Fit Logistic regression
        logreg = LogisticRegression(max_iter=5000, penalty='elasticnet', solver='saga', l1_ratio=0.5)
        logreg.fit(os_data_X, os_data_y)
        #VOIs[ii] = list(ind_var[rfe.ranking_==1])
        VOIs[ii] = list(ind_var[rfe.get_support()])
        # Perform prediction
        y_pred = logreg.predict(X_test)

        # ROC curve
        logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
        ROC_AUC[ii] = logit_roc_auc 
        # Collect True and False Positive rates
        fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
        FPR[ii] = fpr
        TPR[ii] = tpr
    return VOIs, ROC_AUC, FPR, TPR




In [ ]:
def plot_ROC(ROC_AUC, FPR, TPR):
    plt.figure()
    for ii in range(len(ROC_AUC)):
        plt.plot(np.asarray(list(FPR.values()))[ii], np.asarray(list(TPR.values()))[ii], 'k-', alpha=0.5, linewidth=.25)
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC AUC average: {0}, median:{1}'.format(round(np.mean(list(ROC_AUC.values())),3), round(np.median(list(ROC_AUC.values())),3)))
    plt.show()
    
def show_VOI(VOIs):
    from palettable.colorbrewer.qualitative import Pastel1_7
    flat_list = [item for sublist in list(VOIs.values()) for item in sublist]
    unique_VOIs = np.unique(flat_list)
    #unique_VOIs = np.unique(np.ravel(list(VOIs.values())))
    VOI_count = {}
    for ii in range(len(unique_VOIs)):
        VOI_count[unique_VOIs[ii]] = np.sum(np.asarray(flat_list) == unique_VOIs[ii])
    def my_autopct(pct):
        return ('%1.1f%%' % pct) if pct > 5 else ''
    labels = np.asarray(list(VOI_count.keys()))
    labels[list(VOI_count.values()) < (np.sum(list(VOI_count.values()))*0.05)] = ''
    my_circle=plt.Circle( (0,0), 0.7, color='white')
    plt.pie(list(VOI_count.values()), labels=labels, colors=Pastel1_7.hex_colors, textprops={'fontsize': 7}, autopct=my_autopct)
    p=plt.gcf()
    p.gca().add_artist(my_circle)
    plt.show()
    
    

In [ ]:
# Cases only
corr_matrix_cases = corr_matrix[corr_matrix.transpose().columns.isin(np.unique(adata.obs['ind_cov'][adata.obs['SLE status']=='SLE'].tolist()))]
            

In [ ]:
corr_matrix
# Cluster genes in expression/cluster this

# Kidney involvment


In [ ]:
%%capture
VOIs, ROC_AUC, FPR, TPR = logistic_regression(corr_matrix_cases, clincal_var='kidney', n_boot=100)

In [ ]:
show_VOI(VOIs)
plot_ROC(ROC_AUC, FPR, TPR)

# Malar rash

In [ ]:
%%capture
VOIs, ROC_AUC, FPR, TPR = logistic_regression(corr_matrix_cases, clincal_var='acrmalar', n_boot=100)

In [ ]:
show_VOI(VOIs)
plot_ROC(ROC_AUC, FPR, TPR)

# Discoid rash

In [ ]:
%%capture
VOIs, ROC_AUC, FPR, TPR = logistic_regression(corr_matrix_cases, clincal_var='acrdiscoid', n_boot=100)

In [ ]:
show_VOI(VOIs)
plot_ROC(ROC_AUC, FPR, TPR)

# Photosensitivity

In [ ]:
%%capture
VOIs, ROC_AUC, FPR, TPR = logistic_regression(corr_matrix_cases, clincal_var='acrphotosensitiv', n_boot=100)

In [ ]:
show_VOI(VOIs)
plot_ROC(ROC_AUC, FPR, TPR)

# Mucosal ulcers

In [ ]:
%%capture
VOIs, ROC_AUC, FPR, TPR = logistic_regression(corr_matrix_cases, clincal_var='acrmuculcers', n_boot=100)

In [ ]:
show_VOI(VOIs)
plot_ROC(ROC_AUC, FPR, TPR)

# Arthritis

In [ ]:
%%capture
VOIs, ROC_AUC, FPR, TPR = logistic_regression(corr_matrix_cases, clincal_var='acrarthritisc', n_boot=100)

In [ ]:
show_VOI(VOIs)
plot_ROC(ROC_AUC, FPR, TPR)

# Serositis

In [ ]:
%%capture
VOIs, ROC_AUC, FPR, TPR = logistic_regression(corr_matrix_cases, clincal_var='acrserositisc', n_boot=100)

In [ ]:
show_VOI(VOIs)
plot_ROC(ROC_AUC, FPR, TPR)
# Why does RP4. Do cases analysis

# Lupus Nephritis

In [ ]:
%%capture
VOIs, ROC_AUC, FPR, TPR = logistic_regression(corr_matrix_cases, clincal_var='acrlupneph', n_boot=100)

In [ ]:
show_VOI(VOIs)
plot_ROC(ROC_AUC, FPR, TPR)

# Renal involvement

In [ ]:
%%capture
VOIs, ROC_AUC, FPR, TPR = logistic_regression(corr_matrix_cases, clincal_var='acrrenalc', n_boot=100)

In [ ]:
show_VOI(VOIs)
plot_ROC(ROC_AUC, FPR, TPR)

# anti-ANA

In [ ]:
%%capture
VOIs, ROC_AUC, FPR, TPR = logistic_regression(corr_matrix, clincal_var='acranac', n_boot=100)

In [ ]:
show_VOI(VOIs)
plot_ROC(ROC_AUC, FPR, TPR)

# anti-Smith

In [ ]:
%%capture
VOIs, ROC_AUC, FPR, TPR = logistic_regression(corr_matrix, clincal_var='acrantismith', n_boot=100)

In [ ]:
show_VOI(VOIs)
plot_ROC(ROC_AUC, FPR, TPR)

# anti-dsDNA

In [ ]:
%%capture
VOIs, ROC_AUC, FPR, TPR = logistic_regression(corr_matrix_cases, clincal_var='acrantidsdna', n_boot=100)

In [ ]:
show_VOI(VOIs)
plot_ROC(ROC_AUC, FPR, TPR)

# anti-phospholipid antigen

In [ ]:
%%capture
VOIs, ROC_AUC, FPR, TPR = logistic_regression(corr_matrix_cases, clincal_var='acrapla', n_boot=100)

In [ ]:
show_VOI(VOIs)
plot_ROC(ROC_AUC, FPR, TPR)

# Lymphopenia

In [ ]:
%%capture
VOIs, ROC_AUC, FPR, TPR, = logistic_regression(corr_matrix_cases, clincal_var='acrlymphopenia', n_boot=100)

In [ ]:
show_VOI(VOIs)
plot_ROC(ROC_AUC, FPR, TPR)

# Leukopenia

In [ ]:
%%capture
VOIs, ROC_AUC, FPR, TPR = logistic_regression(corr_matrix, clincal_var='acrleukopenia', n_boot=100)

In [ ]:
show_VOI(VOIs)
plot_ROC(ROC_AUC, FPR, TPR)

# Thrombocytopenia

In [ ]:
%%capture
VOIs, ROC_AUC, FPR, TPR = logistic_regression(corr_matrix, clincal_var='acrthrombocyto', n_boot=100)

In [ ]:
show_VOI(VOIs)
plot_ROC(ROC_AUC, FPR, TPR)

# Linear regression on sum of logistic regression predictions

In [ ]:
def run_logreg_bank(corr_matrix, clincal, lupus_var, final_y, split):
    acrdata = clincal.transpose()[clincal.columns.isin(lupus_var)].transpose()

    # Remove people with no clinical variables
    corr_matrix_logreg = corr_matrix[~corr_matrix.transpose().columns.isin(['FLARE002', 'FLARE004', 'FLARE005', 'FLARE006', 
                                                                         'FLARE007', 'FLARE009', 'FLARE011', 'FLARE013', 
                                                                         'FLARE014', 'FLARE015', 'FLARE016', 'FLARE017', 
                                                                         'FLARE018', 'FLARE019', 'FLARE020'])]
    # List of ACR clinical covariates
    clin_var = list(acrdata.columns) 
    # Add clinical covariates to dataframe
    for clin in clin_var:
        corr_matrix_logreg = build_covar(corr_matrix_logreg, keys = [clin])
    # Drop people with incomplete data from dataframe.
    corr_matrix_logreg = corr_matrix_logreg.dropna()
    # ACR sum - dependent variable for linear regression
    Y = corr_matrix_logreg[final_y].values.ravel()
    # Remove dependent variable for linear regression from dataframe and clinical covariates
    corr_matrix_logreg = corr_matrix_logreg.drop(final_y, axis=1)
    clin_var.remove(final_y)

    # Split data
    X_train, X_test, y_linear_train, y_linear_test = train_test_split(corr_matrix_logreg, Y, test_size=split)
    # Generate dictionaries to store outcome of each logistic regression
    VOIs = {}; ROC_AUC = {}; FPR = {}; TPR = {}; Y_logreg_pred ={}
    for lr in range(len(clin_var)):
        try:
            clin_tmp = clin_var.copy()
            print('Clinical Variable: {}'.format(clin_tmp[lr]))
            # Remove all dependent variables except one of interest.
            clin_tmp.remove(clin_tmp[lr])
            X_train_tmp = X_train.drop(clin_tmp, axis=1)
            X_test_tmp  = X_test.drop(clin_tmp, axis=1)
            # Dependent variable for logistic regression
            Y_logreg_train = X_train_tmp[clin_var[lr]].values.ravel()
            Y_logreg_test  = X_test_tmp[clin_var[lr]].values.ravel()
            # Remove dependent varaible for logistic regression from dataframe
            X_train_tmp = X_train_tmp.drop(clin_var[lr], axis=1)
            X_test_tmp = X_test_tmp.drop(clin_var[lr], axis=1)
            # Over sample cases and undersample controls to improve classification
            os = SMOTE(random_state=0)
            os_data_X, os_data_y = os.fit_sample(X_train_tmp, Y_logreg_train)
            # Setup logistic regression
            logreg = LogisticRegression(max_iter=2000)
            # Find optimal parameters
            rfe = RFE(logreg, 3)
            rfe = rfe.fit(os_data_X, os_data_y)
            ind_var = np.asarray(list(os_data_X.columns))

            # Preserve optimate features only from train and test set
            os_data_X = os_data_X[os_data_X.columns[os_data_X.columns.isin(ind_var[rfe.ranking_==1])]]
            X_test_tmp = X_test_tmp[os_data_X.columns[os_data_X.columns.isin(ind_var[rfe.ranking_==1])]]

            # Fit Logistic regression
            logreg = LogisticRegression(max_iter=2000)
            logreg.fit(os_data_X, os_data_y)
            VOIs[clin_var[lr]] = list(ind_var[rfe.ranking_==1])
            # Perform prediction
            y_pred = logreg.predict(X_test_tmp)
            Y_logreg_pred[clin_var[lr]] = y_pred
            # ROC curve
            logit_roc_auc = roc_auc_score(Y_logreg_test, y_pred)
            ROC_AUC[clin_var[lr]] = logit_roc_auc 
            # Collect True and False Positive rates
            fpr, tpr, thresholds = roc_curve(Y_logreg_test, logreg.predict_proba(X_test_tmp)[:,1])
            FPR[clin_var[lr]] = fpr
            TPR[clin_var[lr]] = tpr
            print('ROC_AOC: {}'.format(ROC_AUC[clin_var[lr]]))
            print('Variables: {}'.format(VOIs[clin_var[lr]]))
        except:
            print('Unable to model variable: {}'.format(clin_tmp[lr]))
            continue
    return VOIs, Y_logreg_pred, y_linear_test, ROC_AUC, FPR, TPR



# ACR Sum prediction

In [ ]:
final_y = 'acrcsum'
# Keep only ACR data
lupus_var = ['acrarthritisc','acrantidsdna', 'acrantismith', 'acrlupneph', 'acranac', final_y]
VOIs, Y_logreg_pred, y_linear_test, ROC_AUC, FPR, TPR = run_logreg_bank(corr_matrix, clincal, lupus_var, final_y, split=0.2)

model = sm.OLS(pd.DataFrame.from_dict(Y_logreg_pred).transpose().sum(axis=0).values, y_linear_test)
results = model.fit()
print(results.summary())
print('Parameters: ', results.params)
sns.regplot(pd.DataFrame.from_dict(Y_logreg_pred).transpose().sum(axis=0).values, y_linear_test)
plt.xlabel('Prediction')
plt.ylabel(final_y)


# SLEDAI prediction

In [ ]:
final_y = 'sledaiscore'
# Keep only ACR data
lupus_var = ['acrarthritisc','acrantidsdna', 'acrantismith', 'acrlupneph', 'acranac', final_y]
VOIs, Y_logreg_pred, y_linear_test, ROC_AUC, FPR, TPR = run_logreg_bank(corr_matrix, clincal, lupus_var, final_y, split=0.2)

model = sm.OLS(pd.DataFrame.from_dict(Y_logreg_pred).transpose().sum(axis=0).values, y_linear_test)
results = model.fit()
print(results.summary())
print('Parameters: ', results.params)
sns.regplot(pd.DataFrame.from_dict(Y_logreg_pred).transpose().sum(axis=0).values, y_linear_test)
plt.xlabel('Prediction')
plt.ylabel(final_y)


In [ ]:
model = sm.OLS(clincal['acrcsum'].values, clincal['sledaiscore'].values)
results = model.fit()
print(results.summary())
print('Parameters: ', results.params)
sns.regplot(clincal['acrcsum'].values, clincal['sledaiscore'].values)
plt.xlabel('ACR sum')
plt.ylabel('SLEDAI')

# SLIC prediction

In [ ]:
final_y = 'SLIC score'
# Keep only ACR data
lupus_var = ['acrarthritisc','acrantidsdna', 'acrantismith', 'acrlupneph', 'acranac', final_y]
VOIs, Y_logreg_pred, y_linear_test, ROC_AUC, FPR, TPR = run_logreg_bank(corr_matrix, clincal, lupus_var, final_y, split=0.2)

model = sm.OLS(pd.DataFrame.from_dict(Y_logreg_pred).transpose().sum(axis=0).values, y_linear_test)
results = model.fit()
print(results.summary())
print('Parameters: ', results.params)
sns.regplot(pd.DataFrame.from_dict(Y_logreg_pred).transpose().sum(axis=0).values, y_linear_test)
plt.xlabel('Prediction')
plt.ylabel(final_y)


In [ ]:
model = sm.OLS(clincal['acrcsum'].values, clincal['SLIC score'].values)
results = model.fit()
print(results.summary())
print('Parameters: ', results.params)
sns.regplot(clincal['acrcsum'].values, clincal['SLIC score'].values)
plt.xlabel('ACR sum')
plt.ylabel('SLIC score')

# SLAQ prediction

In [ ]:
final_y = 'SLAQ score'
# Keep only ACR data
lupus_var = ['acrarthritisc','acrantidsdna', 'acrantismith', 'acrlupneph', 'acranac', final_y]
VOIs, Y_logreg_pred, y_linear_test, ROC_AUC, FPR, TPR = run_logreg_bank(corr_matrix, clincal, lupus_var, final_y, split=0.2)

model = sm.OLS(pd.DataFrame.from_dict(Y_logreg_pred).transpose().sum(axis=0).values, y_linear_test)
results = model.fit()
print(results.summary())
print('Parameters: ', results.params)
sns.regplot(pd.DataFrame.from_dict(Y_logreg_pred).transpose().sum(axis=0).values, y_linear_test)
plt.xlabel('Prediction')
plt.ylabel(final_y)


In [ ]:
model = sm.OLS(clincal['acrcsum'].values, clincal['SLAQ score'].values)
results = model.fit()
print(results.summary())
print('Parameters: ', results.params)
sns.regplot(clincal['acrcsum'].values, clincal['SLAQ score'].values)
plt.xlabel('ACR sum')
plt.ylabel('SLAQ score')

# ROC curves comparing disease status and associated scale

### Patients may have SLE but exhibit no SLE symptoms at the time of blood collection

In [ ]:
acrdata = clincal.transpose()[clincal.columns.isin(lupus_var)].transpose()
corr_matrix_ROC = pd.DataFrame(index=list(corr_matrix.index))
# Remove people with no clinical variables
corr_matrix_ROC = corr_matrix_ROC[~corr_matrix_ROC.transpose().columns.isin(['FLARE002', 'FLARE004', 'FLARE005', 'FLARE006', 
                                                                     'FLARE007', 'FLARE009', 'FLARE011', 'FLARE013', 
                                                                     'FLARE014', 'FLARE015', 'FLARE016', 'FLARE017', 
                                                                     'FLARE018', 'FLARE019', 'FLARE020'])]
# List of ACR clinical covariates
clin_var = list(acrdata.columns)
# Add clinical covariates to dataframe
for clin in clin_var:
    corr_matrix_ROC = build_covar(corr_matrix_ROC, keys = ['Disease_status', 'acrcsum', 'sledaiscore', 'SLIC score', 'SLAQ score', 'sleactivity'])
# Drop people with incomplete data from dataframe.
corr_matrix_ROC = corr_matrix_ROC.dropna()
# ACR sum - dependent variable for linear regression
Y = corr_matrix_ROC['Disease_status'].values.ravel()
# Remove dependent variable for linear regression from dataframe and clinical covariates
corr_matrix_ROC = corr_matrix_ROC.drop('Disease_status', axis=1)
clin_var.remove(final_y)

# Turn scales into diagnosis outcomes based study cutoffs
corr_matrix_ROC['acrcsum'] = ((corr_matrix_ROC['acrcsum'] >=4)*1).values
corr_matrix_ROC['sledaiscore'] = ((corr_matrix_ROC['sledaiscore'] >=3)*1).values
corr_matrix_ROC['SLIC score'] = ((corr_matrix_ROC['SLIC score'] >=4)*1).values
corr_matrix_ROC['SLAQ score'] = ((corr_matrix_ROC['SLAQ score'] >=4)*1).values
corr_matrix_ROC['sleactivity'] = ((corr_matrix_ROC['sleactivity'] >=3)*1).values

In [ ]:
scales = list(corr_matrix_ROC.columns)
for ii in range(corr_matrix_ROC.shape[1]):
    # ROC curve
    logit_roc_auc = roc_auc_score(Y, corr_matrix_ROC[scales[ii]].values)
    # Collect True and False Positive rates
    fpr, tpr, thresholds = roc_curve(Y, corr_matrix_ROC[scales[ii]].values)

    plt.figure()
    plt.plot(fpr, tpr, 'k-', alpha=1, linewidth=2)
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC AUC: {0}, for {1} scale'.format(round(logit_roc_auc,3), scales[ii]))
    plt.show()


# Two Stage Logistic-Logistic model for prediction of clinical scales

In [ ]:
def show_important_clinical_features(coef, labels):
    from palettable.colorbrewer.qualitative import Pastel1_7
    vals = np.asarray(list(coef.values())).mean(axis=0)/np.sum(np.asarray(list(coef.values())).mean(axis=0))
    def my_autopct(pct):
        return ('%1.1f%%' % pct) if pct > 0 else ''
    my_circle=plt.Circle( (0,0), 0.7, color='white')
    plt.pie(vals, labels=labels, colors=Pastel1_7.hex_colors, textprops={'fontsize': 7}, autopct=my_autopct)
    p=plt.gcf()
    p.gca().add_artist(my_circle)
    plt.show()

# ACR diagnosis

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
final_y = 'acrcsum'
# Keep only ACR data
lupus_var = ['acrarthritisc','acrantidsdna', 'acrantismith', 'acrlupneph', 'acranac', final_y]
# Variables of interest
VOIs_gene = {}; VOIs = {}; ROC_AUC = {}; FPR = {}; TPR = {}; R2 = {}; coef = {}
model='LogReg'
for ii in range(50):
    #try:
    VOIs_gene[ii], Y_logreg_pred, y_linear_test, ROC_AUC_, FPR_, TPR_ = run_logreg_bank(corr_matrix, clincal, lupus_var, final_y, split=0.6)
    if model == 'LogReg':
        # Convert ordinal scale to binary SLE Diagnosis or not
        y_linear_test = (y_linear_test>=4)*1
    logoutput = pd.DataFrame.from_dict(Y_logreg_pred)
    X_train, X_test, y_linear_train, y_linear_test_2 = train_test_split(logoutput, y_linear_test, test_size=0.2)
    if model == 'LogReg':
        os = SMOTE(random_state=0)
        X_train, y_linear_train = os.fit_sample(X_train, y_linear_train)
        # Setup logistic regression
        logreg = LogisticRegression(max_iter=1000)
    if model == 'Linear':
        logreg = LinearRegression()
    # Find optimal parameters
    rfe = RFE(logreg, 5)
    rfe = rfe.fit(X_train, y_linear_train)
    ind_var = np.asarray(list(X_train.columns))
    # Preserve optimate features only from train and test set
    X_train = X_train[X_train.columns[X_train.columns.isin(ind_var[rfe.ranking_==1])]]
    X_test = X_test[X_train.columns[X_train.columns.isin(ind_var[rfe.ranking_==1])]]
    if model == 'LogReg':
        clf = LogisticRegression(max_iter=1000)
    if model == 'Linear':
        clf = LinearRegression()
    clf.fit(X_train, y_linear_train)
    print('VOIs: {}'.format(list(ind_var[rfe.ranking_==1])))
    VOIs[ii] = list(ind_var[rfe.ranking_==1])
    model_pred = clf.predict(X_test)
    coef[ii] = clf.coef_.tolist()[0]
    labels = list(X_test.columns)
    if model == 'LogReg':
        # ROC curve
        logit_roc_auc = roc_auc_score(y_linear_test_2, clf.predict(X_test))
        ROC_AUC[ii] = logit_roc_auc 
        # Collect True and False Positive rates
        fpr, tpr, thresholds = roc_curve(y_linear_test_2, clf.predict_proba(X_test)[:,1])
        FPR[ii] = fpr
        TPR[ii] = tpr
        print('2nd stage ROC AUC: {}'.format(logit_roc_auc))
    if model == 'Linear':
        R2[ii] = clf.score(X_test, y_linear_test_2)
        print('R squared: {}'.format(clf.score(X_test, y_linear_test_2)))
#except:
    #    continue


In [ ]:
# Gene signatures underlying diagnosis of SLE with ACR scale
VOI_genes = [list(np.ravel(list(VOIs_gene[ii].values()))) for ii in VOIs_gene]
VOIs = [item for sublist in VOI_genes for item in sublist]
VOIs = {'VOIs':VOIs  }
show_VOI(VOIs)
plot_ROC(ROC_AUC, FPR, TPR) # split 0.6
show_important_clinical_features(coef, labels)
# Reach out to Yun Song

# Cohort diagnosis (based on SLE or Healthy label)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
final_y = 'Disease_status'
# Keep only ACR data
lupus_var = ['acrarthritisc','acrantidsdna', 'acrantismith', 'acrlupneph', 'acranac', final_y]
# Variables of interest
VOIs_gene = {}; VOIs = {}; ROC_AUC = {}; FPR = {}; TPR = {}; R2 = {}; 
model='LogReg'
for ii in range(50):
    #try:
    VOIs_gene[ii], Y_logreg_pred, y_linear_test, ROC_AUC_, FPR_, TPR_ = run_logreg_bank(corr_matrix, clincal, lupus_var, final_y, split=0.6)
    logoutput = pd.DataFrame.from_dict(Y_logreg_pred)
    X_train, X_test, y_linear_train, y_linear_test_2 = train_test_split(logoutput, y_linear_test, test_size=0.2)
    if model == 'LogReg':
        os = SMOTE(random_state=0)
        X_train, y_linear_train = os.fit_sample(X_train, y_linear_train)
        # Setup logistic regression
        logreg = LogisticRegression(max_iter=1000)
    if model == 'Linear':
        logreg = LinearRegression()
    # Find optimal parameters
    rfe = RFE(logreg, 5)
    rfe = rfe.fit(X_train, y_linear_train)
    ind_var = np.asarray(list(X_train.columns))
    # Preserve optimate features only from train and test set
    X_train = X_train[X_train.columns[X_train.columns.isin(ind_var[rfe.ranking_==1])]]
    X_test = X_test[X_train.columns[X_train.columns.isin(ind_var[rfe.ranking_==1])]]
    if model == 'LogReg':
        clf = LogisticRegression(max_iter=1000)
    if model == 'Linear':
        clf = LinearRegression()
    clf.fit(X_train, y_linear_train)
    print('VOIs: {}'.format(list(ind_var[rfe.ranking_==1])))
    VOIs[ii] = list(ind_var[rfe.ranking_==1])
    coef[ii] = clf.coef_.tolist()[0]
    labels = list(X_test.columns)
    model_pred = clf.predict(X_test)
    if model == 'LogReg':
        # ROC curve
        logit_roc_auc = roc_auc_score(y_linear_test_2, clf.predict(X_test))
        ROC_AUC[ii] = logit_roc_auc 
        # Collect True and False Positive rates
        fpr, tpr, thresholds = roc_curve(y_linear_test_2, clf.predict_proba(X_test)[:,1])
        FPR[ii] = fpr
        TPR[ii] = tpr
        print('2nd stage ROC AUC: {}'.format(logit_roc_auc))
    if model == 'Linear':
        R2[ii] = clf.score(X_test, y_linear_test_2)
        print('R squared: {}'.format(clf.score(X_test, y_linear_test_2)))
#except:
    #    continue


In [ ]:
# Gene signatures underlying diagnosis of SLE with ACR scale
VOI_genes = [list(np.ravel(list(VOIs_gene[ii].values()))) for ii in VOIs_gene]
VOIs = [item for sublist in VOI_genes for item in sublist]
VOIs = {'VOIs':VOIs}
show_VOI(VOIs)
plot_ROC(ROC_AUC, FPR, TPR) # split 0.6
show_important_clinical_features(coef, labels)

# SLEDAI Diagnosis

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
final_y = 'acrcsum'
# Keep only ACR data
lupus_var = ['acrarthritisc','acrantidsdna', 'acrantismith', 'acrlupneph', 'acranac', final_y]
# Variables of interest
VOIs_gene = {}; VOIs = {}; ROC_AUC = {}; FPR = {}; TPR = {}; R2 = {}; 
model='LogReg'
for ii in range(50):
    #try:
    VOIs_gene[ii], Y_logreg_pred, y_linear_test, ROC_AUC_, FPR_, TPR_ = run_logreg_bank(corr_matrix, clincal, lupus_var, final_y, split=0.6)
    if model == 'LogReg':
        # Convert ordinal scale to binary SLE Diagnosis or not
        y_linear_test = (y_linear_test>=4)*1
    logoutput = pd.DataFrame.from_dict(Y_logreg_pred)
    X_train, X_test, y_linear_train, y_linear_test_2 = train_test_split(logoutput, y_linear_test, test_size=0.2)
    if model == 'LogReg':
        os = SMOTE(random_state=0)
        X_train, y_linear_train = os.fit_sample(X_train, y_linear_train)
        # Setup logistic regression
        logreg = LogisticRegression(max_iter=1000)
    if model == 'Linear':
        logreg = LinearRegression()
    # Find optimal parameters
    rfe = RFE(logreg, 5)
    rfe = rfe.fit(X_train, y_linear_train)
    ind_var = np.asarray(list(X_train.columns))
    # Preserve optimate features only from train and test set
    X_train = X_train[X_train.columns[X_train.columns.isin(ind_var[rfe.ranking_==1])]]
    X_test = X_test[X_train.columns[X_train.columns.isin(ind_var[rfe.ranking_==1])]]
    if model == 'LogReg':
        clf = LogisticRegression(max_iter=1000)
    if model == 'Linear':
        clf = LinearRegression()
    clf.fit(X_train, y_linear_train)
    print('VOIs: {}'.format(list(ind_var[rfe.ranking_==1])))
    VOIs[ii] = list(ind_var[rfe.ranking_==1])
    coef[ii] = clf.coef_.tolist()[0]
    labels = list(X_test.columns)
    model_pred = clf.predict(X_test)
    if model == 'LogReg':
        # ROC curve
        logit_roc_auc = roc_auc_score(y_linear_test_2, clf.predict(X_test))
        ROC_AUC[ii] = logit_roc_auc 
        # Collect True and False Positive rates
        fpr, tpr, thresholds = roc_curve(y_linear_test_2, clf.predict_proba(X_test)[:,1])
        FPR[ii] = fpr
        TPR[ii] = tpr
        print('2nd stage ROC AUC: {}'.format(logit_roc_auc))
    if model == 'Linear':
        R2[ii] = clf.score(X_test, y_linear_test_2)
        print('R squared: {}'.format(clf.score(X_test, y_linear_test_2)))
#except:
    #    continue


In [ ]:
# Gene signatures underlying diagnosis of SLE with ACR scale
VOI_genes = [list(np.ravel(list(VOIs_gene[ii].values()))) for ii in VOIs_gene]
VOIs = [item for sublist in VOI_genes for item in sublist]
VOIs = {'VOIs':VOIs}
show_VOI(VOIs)
plot_ROC(ROC_AUC, FPR, TPR) # split 0.6
show_important_clinical_features(coef, labels)

# SLAQ Diagnosis

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
final_y = 'SLAQ score'
# Keep only ACR data
lupus_var = ['acrarthritisc','acrantidsdna', 'acrantismith', 'acrlupneph', 'acranac', final_y]
# Variables of interest
VOIs_gene = {}; VOIs = {}; ROC_AUC = {}; FPR = {}; TPR = {}; R2 = {}; 
model='LogReg'
for ii in range(50):
    #try:
    VOIs_gene[ii], Y_logreg_pred, y_linear_test, ROC_AUC_, FPR_, TPR_ = run_logreg_bank(corr_matrix, clincal, lupus_var, final_y, split=0.6)
    if model == 'LogReg':
        # Convert ordinal scale to binary SLE Diagnosis or not
        y_linear_test = (y_linear_test>=4)*1
    logoutput = pd.DataFrame.from_dict(Y_logreg_pred)
    X_train, X_test, y_linear_train, y_linear_test_2 = train_test_split(logoutput, y_linear_test, test_size=0.2)
    if model == 'LogReg':
        os = SMOTE(random_state=0)
        X_train, y_linear_train = os.fit_sample(X_train, y_linear_train)
        # Setup logistic regression
        logreg = LogisticRegression(max_iter=1000)
    if model == 'Linear':
        logreg = LinearRegression()
    # Find optimal parameters
    rfe = RFE(logreg, 5)
    rfe = rfe.fit(X_train, y_linear_train)
    ind_var = np.asarray(list(X_train.columns))
    # Preserve optimate features only from train and test set
    X_train = X_train[X_train.columns[X_train.columns.isin(ind_var[rfe.ranking_==1])]]
    X_test = X_test[X_train.columns[X_train.columns.isin(ind_var[rfe.ranking_==1])]]
    if model == 'LogReg':
        clf = LogisticRegression(max_iter=1000)
    if model == 'Linear':
        clf = LinearRegression()
    clf.fit(X_train, y_linear_train)
    print('VOIs: {}'.format(list(ind_var[rfe.ranking_==1])))
    VOIs[ii] = list(ind_var[rfe.ranking_==1])
    coef[ii] = clf.coef_.tolist()[0]
    labels = list(X_test.columns)
    model_pred = clf.predict(X_test)
    if model == 'LogReg':
        # ROC curve
        logit_roc_auc = roc_auc_score(y_linear_test_2, clf.predict(X_test))
        ROC_AUC[ii] = logit_roc_auc 
        # Collect True and False Positive rates
        fpr, tpr, thresholds = roc_curve(y_linear_test_2, clf.predict_proba(X_test)[:,1])
        FPR[ii] = fpr
        TPR[ii] = tpr
        print('2nd stage ROC AUC: {}'.format(logit_roc_auc))
    if model == 'Linear':
        R2[ii] = clf.score(X_test, y_linear_test_2)
        print('R squared: {}'.format(clf.score(X_test, y_linear_test_2)))
#except:
    #    continue


In [ ]:
# Gene signatures underlying diagnosis of SLE with ACR scale
VOI_genes = [list(np.ravel(list(VOIs_gene[ii].values()))) for ii in VOIs_gene]
VOIs = [item for sublist in VOI_genes for item in sublist]
VOIs = {'VOIs':VOIs}
show_VOI(VOIs)
plot_ROC(ROC_AUC, FPR, TPR) # split 0.6
show_important_clinical_features(coef, labels)

# SLIC Diagnosis

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
final_y = 'SLIC score'
# Keep only ACR data
lupus_var = ['acrarthritisc','acrantidsdna', 'acrantismith', 'acrlupneph', 'acranac', final_y]
# Variables of interest
VOIs_gene = {}; VOIs = {}; ROC_AUC = {}; FPR = {}; TPR = {}; R2 = {}; 
model='LogReg'
for ii in range(50):
    #try:
    VOIs_gene[ii], Y_logreg_pred, y_linear_test, ROC_AUC_, FPR_, TPR_ = run_logreg_bank(corr_matrix, clincal, lupus_var, final_y, split=0.6)
    if model == 'LogReg':
        # Convert ordinal scale to binary SLE Diagnosis or not
        y_linear_test = (y_linear_test>=4)*1
    logoutput = pd.DataFrame.from_dict(Y_logreg_pred)
    X_train, X_test, y_linear_train, y_linear_test_2 = train_test_split(logoutput, y_linear_test, test_size=0.2)
    if model == 'LogReg':
        os = SMOTE(random_state=0)
        X_train, y_linear_train = os.fit_sample(X_train, y_linear_train)
        # Setup logistic regression
        logreg = LogisticRegression(max_iter=1000)
    if model == 'Linear':
        logreg = LinearRegression()
    # Find optimal parameters
    rfe = RFE(logreg, 5)
    rfe = rfe.fit(X_train, y_linear_train)
    ind_var = np.asarray(list(X_train.columns))
    # Preserve optimate features only from train and test set
    X_train = X_train[X_train.columns[X_train.columns.isin(ind_var[rfe.ranking_==1])]]
    X_test = X_test[X_train.columns[X_train.columns.isin(ind_var[rfe.ranking_==1])]]
    if model == 'LogReg':
        clf = LogisticRegression(max_iter=1000)
    if model == 'Linear':
        clf = LinearRegression()
    clf.fit(X_train, y_linear_train)
    print('VOIs: {}'.format(list(ind_var[rfe.ranking_==1])))
    VOIs[ii] = list(ind_var[rfe.ranking_==1])
    coef[ii] = clf.coef_.tolist()[0]
    labels = list(X_test.columns)
    model_pred = clf.predict(X_test)
    if model == 'LogReg':
        # ROC curve
        logit_roc_auc = roc_auc_score(y_linear_test_2, clf.predict(X_test))
        ROC_AUC[ii] = logit_roc_auc 
        # Collect True and False Positive rates
        fpr, tpr, thresholds = roc_curve(y_linear_test_2, clf.predict_proba(X_test)[:,1])
        FPR[ii] = fpr
        TPR[ii] = tpr
        print('2nd stage ROC AUC: {}'.format(logit_roc_auc))
    if model == 'Linear':
        R2[ii] = clf.score(X_test, y_linear_test_2)
        print('R squared: {}'.format(clf.score(X_test, y_linear_test_2)))
#except:
    #    continue


In [ ]:
# Gene signatures underlying diagnosis of SLE with ACR scale
VOI_genes = [list(np.ravel(list(VOIs_gene[ii].values()))) for ii in VOIs_gene]
VOIs = [item for sublist in VOI_genes for item in sublist]
VOIs = {'VOIs':VOIs}
show_VOI(VOIs)
plot_ROC(ROC_AUC, FPR, TPR) # split 0.6
show_important_clinical_features(coef, labels)

# Feature plots for Untreated Treated pairs

In [ ]:
%%capture
pairs = ['FLARE004', 'FLARE009', 'FLARE011', 'FLARE013', 'FLARE016', 'FLARE003', 'FLARE002', 'FLARE008']
colors = ["#000000", "#FF0000"]
for pair in pairs:
    bdata = adata[adata.obs['ind_cov']==pair]
    print(pair)
    sc.pl.umap(bdata, size=10, show=True, color='disease_cov', save=str('TREATED_UNTREATED_PAIR'+ pair + '.disease.png'), palette=colors)
    plt.show() 

# Feature plots for marker genes

In [ ]:
# Atypical memory B cells
sc.pl.umap(adata, color=['CR2', 'CD27', 'FCRL5', 'ITGAX', 'NKG7'], show=False, save='.AtypicalBgenes.png')

In [ ]:
## Platelet effect
sc.pl.umap(adata, color=["PF4", "SDPR", "GNG11", "PPBP"], size=3, show=False, save='.platelet.png')

In [ ]:
## Cycling
sc.pl.umap(adata, color=["KIAA0101","STMN1","TK1","MKI67"], size=3, show=False, save='.cycling.png')

In [ ]:
## Macrophages
sc.pl.umap(adata, color=['CD163', 'HLA-DRB1', 'C1QA', 'IFITM3'], size=3, show=False, save=".MACRO.png")

In [ ]:
## CD4, CD8 and NK axis
sc.pl.umap(adata, color=["CD3D","CD8A","CD4","NCAM1", "FCGR3A", 'NKG7', 'IFNG', 'GZMB', 'PRF1'],size=3, show=True, save=".TNK.png")
# Two NK sub populations: CD56high/lowCD16high/low, CD56lowCD16high
# https://www.frontiersin.org/files/Articles/162361/fimmu-06-00567-HTML/image_m/fimmu-06-00567-g001.jpg

In [ ]:
## Memory vs. Helper vs. Naive
sc.pl.umap(adata, color=["CCR7","IL7R","S100A4","CD58", 'FAS', 'IL2RA'],show=True, size=3, save=".TMemThTNaive.png")

In [ ]:
## T-Reg
sc.pl.umap(adata, color=["FOXP3","TNFRSF4","ENTPD1","CCR10"],size=3, show=False, save=".TREG.png")

In [ ]:
## Y chromosome gender effect
sc.pl.umap(adata, color=["DDX3Y", "RPS4Y1", "FHIT","TRAT1"], size =3, show=False, save=".Y.png")

In [ ]:
## B cells, plasmablasts and pdcs
sc.pl.umap(adata, color=['BTLA', 'P2RY8',"MZB1", "CD19", "CD79A", 'MS4A1', 'FCRL5', 'IL6', 'CR2'],size=3, show=False, save=".B.png")


# Protein Log2 FC change analysis 

In [ ]:
# Subset data to contain only lupus cohort 3 +- flare and compute log2 FC change for total PBMC 
# and each cell type population. Correlate with clinical variables and scales in addition to proportion cell types.
# Possibly use log2FC genes and add them into correlation plot without recomputing them.


# Gene Ontology analysis for each cell type

In [ ]:
from __future__ import print_function
from Bio import Entrez
from goatools.base import download_go_basic_obo
from goatools.base import download_ncbi_associations
from goatools.obo_parser import GODag
from goatools.anno.genetogo_reader import Gene2GoReader
from goatools.test_data.genes_NCBI_9606_ProteinCoding import GENEID2NT as GeneID2nt_hsa
from goatools.goea.go_enrichment_ns import GOEnrichmentStudyNS
from goatools.godag_plot import plot_gos, plot_results, plot_goid2goobj
# Store gene summaries
summary_dict = {}
def getgeneids(gene_list):
    Entrez.email = "richard.perez@ucsf.edu"
    # Store symbol and id
    geneid2symbol = {}
    for genesymbol in gene_list:
        try:
            handle = Entrez.esearch(db="gene", term=str(genesymbol + "[GENE] AND Homo"))
            record = Entrez.read(handle)
            geneid2symbol[int(record['IdList'][0])] = genesymbol
            handle.close()
        except:
            continue
    return geneid2symbol

def id2symbol(GENES, celltypes, cdata):
    for geneID in GENES:
        Entrez.email = "richard.perez@ucsf.edu"
        id_list= [str(geneID)]
        request = Entrez.epost("gene",id=",".join(id_list))
        result = Entrez.read(request)
        webEnv = result["WebEnv"]
        queryKey = result["QueryKey"]
        data = Entrez.esummary(db="gene", webenv=webEnv, query_key=queryKey)
        annotations = Entrez.read(data)
        symbol  = annotations['DocumentSummarySet']['DocumentSummary'][0]['Name']
        summary = annotations['DocumentSummarySet']['DocumentSummary'][0]['Summary']
        print(symbol); print(summary)
        ind_perc = plot_gene_expression(cdata, celltypes, symbol)
    return symbol, summary

def id2symbolfast(gene_id):
    Entrez.email = "richard.perez@ucsf.edu"
    id_list= [str(gene_id)]
    request = Entrez.epost("gene",id=",".join(id_list))
    result = Entrez.read(request)
    webEnv = result["WebEnv"]
    queryKey = result["QueryKey"]
    data = Entrez.esummary(db="gene", webenv=webEnv, query_key=queryKey)
    annotations = Entrez.read(data)
    symbol  = annotations['DocumentSummarySet']['DocumentSummary'][0]['Name']
    summary = annotations['DocumentSummarySet']['DocumentSummary'][0]['Summary']
    return symbol, summary

def replace_id_with_symbol(results, summary_dict):
    for ll in range(len(results['study_items'])):
        study_item = np.asarray(results['study_items'][ll].split(','), dtype=int)
        # Keep all gene summaries
        for ii in range(len(study_item)):
            symbol, summary = id2symbolfast(study_item[ii])
            summary_dict[symbol] = summary
        # Replace gene ID with gene symbol
        study_item = [id2symbolfast(study_item[ii])[0] for ii in range(len(study_item))]
        results['study_items'][ll]= study_item
    return results, summary_dict

def run_GO(gene_list, cutoff, title):
    obo_fname = download_go_basic_obo()
    fin_gene2go = download_ncbi_associations()
    obodag = GODag("go-basic.obo")

    # Read NCBI's gene2go. Store annotations in a list of namedtuples
    objanno = Gene2GoReader(fin_gene2go, taxids=[9606])

    # Get namespace2association where:
    #    namespace is:
    #        BP: biological_process               
    #        MF: molecular_function
    #        CC: cellular_component
    #    assocation is a dict:
    #        key: NCBI GeneID
    #        value: A set of GO IDs associated with that gene
    ns2assoc = objanno.get_ns2assc()

    for nspc, id2gos in ns2assoc.items():
        print("{NS} {N:,} annotated human genes".format(NS=nspc, N=len(id2gos)))

    goeaobj = GOEnrichmentStudyNS(
            GeneID2nt_hsa.keys(), # List of human protein-coding genes
            ns2assoc, # geneid/GO associations
            obodag, # Ontologies
            propagate_counts = False,
            alpha = cutoff, # default significance cut-off
            methods = ['fdr_bh']) # defult multipletest correction method

    geneid2symbol = getgeneids(gene_list)
    # 'p_' means "pvalue". 'fdr_bh' is the multiple test method
    geneids_study = geneid2symbol.keys()
    goea_results_all = goeaobj.run_study(geneids_study)
    goea_results_sig = [r for r in goea_results_all if r.p_fdr_bh < cutoff]
    '''
    This plot contains GOEA results:

    GO terms colored by P-value:
    pval < 0.005 (light red)
    pval < 0.01 (light orange)
    pval < 0.05 (yellow)
    pval > 0.05 (grey) Study terms that are not statistically significant
    GO terms with study gene counts printed. e.g., "32 genes"
    '''
    plot_results(title+"{NS}.png", goea_results_sig, id2symbol=geneid2symbol, study_items=20, items_p_line=5)
    goeaobj.wr_xlsx(title+".xlsx", goea_results_sig, id2symbol=geneid2symbol)
    return geneid2symbol, goea_results_sig

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
ct_groups = np.unique(adata.obs['ct_cov'].tolist())
for ii in range(len(ct_groups)):
    try:
        ddata = adata[adata.obs['ct_cov']==ct_groups[ii]]
        sc.tl.rank_genes_groups(ddata, groupby='SLE status',groups=['Healthy', 'SLE'], reference='Healthy', n_genes=len(ddata.raw.var_names))
        sc.pl.rank_genes_groups(ddata, n_genes=25, save='{}_SLEvsHealthy.png'.format(ct_groups[ii]))
        genes = pd.DataFrame(ddata.uns['rank_genes_groups']['names'])
        scores = pd.DataFrame(ddata.uns['rank_genes_groups']['scores'])
        gene_list = list(np.ravel(genes[scores.values>(scores.mean()+(scores.std()*3))[0]].values.tolist()))
        cutoff = 0.001
        title = str(ct_groups[ii])+'_Lupus_Study'
        gene_list = list(np.ravel(genes[scores.values>(scores.mean()+(scores.std()*3))[0]].values.tolist()))
        run_GO(gene_list,cutoff,title)
        results = pd.read_excel(title+".xlsx")
        results, summary_dict = replace_id_with_symbol(results, summary_dict)
        print(ct_groups[ii])
        print('Number of genes 3 standard deviations above mean: {}'.format(np.sum(scores.values>(scores.mean()+(scores.std()*3))[0])))
        display(results)
    except:
        continue